In [ ]:
import os
import sys
import logging
import itertools
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
from typing import Dict, Any, List, Generator

In [ ]:
# package imports
from vsim import electorate, candidates, simulation, voting_system

# Analysing proportional voting systems

We'll be using the case study of the Swedish parlimentary election systems, namely that there exists n-parties, voters vote for the preferred option (using the distance method we described earlier). The difference is the "winner" is not as well defined. 

In [3]:
# setup helper for generating experiments (basically all sets of params to run through sim)
def generate_experiment_matrix(experiments: Dict[str, List[Any]]):
    return (dict(zip(experiments, x)) for x in itertools.product(*experiments.values()))

columns=[
    "electorate_size",
    "clusters",
    "cluster_std" 
    "issues",
    "candidates",
    "electoral_system",
    "apathy_prob",
    "fairness"
    "seats",
    "min_share_threshold"
]

experiments = {
    "electorate_size": [10_000, 25_000, 50_000],
    "clusters": [1, 2, 5, 10],
    "cluster_std": [1, 2, 3],
    "electoral_system": ["proportional"],
    "apathy_prob": [0.0, 0.01, 0.05],
    "candidates": [2, 4, 5, 9], # akin to parties in the proportional case
    "issues": [2, 5, 10, 25],
    "seats": [150, 179, 349, 736], # dutch, danish, swedish, german parliment(s) seats
    "min_share_threshold": [0, 0.04, 0.10] # dutch, swedish, and turkish thresholds
}

records = []
columns = [*experiments.keys(), "fairness"]
experiment_matrix = list(generate_experiment_matrix(experiments))[:10]

for params in tqdm(experiment_matrix):

    # setup objects
    voters = electorate.setup_electorate(**params)
    parties = candidates.setup_candidates(**params, electorate=voters)
    system = voting_system.setup_voting_system(name=params["electoral_system"], **params)

    # simulate elections
    sim = simulation.VotingSimulator(electorate=voters, candidates=parties, system=system, log=logging.getLogger())

    result = sim.run()
    result_record = { "fairness": round(result.measured_fairness, 3), **params }
    result_record = {k: v for k, v in result_record.items() if k in columns}
    records.append(result_record)

  0%|                                           | 0/10 [00:00<?, ?it/s]


TypeError: Axis must be specified when shapes of a and weights differ.

In [ ]:
results_df = pd.DataFrame.from_records(records)
results_df

##### Store data or read from already performed experiment files

In [ ]:
filepath = "data/proportional-vs-proportional-df5ec973-c180-40ef-a410-1490716cd4e2.csv"